In [1]:
import numpy as np
import pandas as pd
import chart_studio.plotly as py

import matplotlib.pyplot as plt
import datetime

# df_covid 전처리 (2019-01-01 ~ 2021-07-31)
df_covid = pd.read_csv('002_2.COVID-19.csv')

df_covid['DATE']= df_covid['DATE'].astype('str')
df_covid['DATE'] = pd.to_datetime(df_covid['DATE'])

df_covid.rename(columns= {'ID' : 'COUNT'}, inplace=True)
df_covid = df_covid.groupby(['DATE']).count()
df_covid.insert(0, 'date', df_covid.index)

df_covid = df_covid[{'date', 'COUNT'}]
df_covid



,date,COUNT
DATE,,
2020-01-24,2020-01-24,1
2020-01-30,2020-01-30,3
2020-01-31,2020-01-31,3
2020-02-02,2020-02-02,1
2020-02-05,2020-02-05,2
...,...,...
2021-08-27,2021-08-27,569
2021-08-28,2021-08-28,517
2021-08-29,2021-08-29,443


In [2]:

#df_subway 전처리 (2019-01-01 ~ 2021-07-31)
df_subway = pd.read_csv('002_1.DAILY_SUBWAY_INFO.csv')
df_subway['date']= df_subway['date'].astype('str')
df_subway['date'] = pd.to_datetime(df_subway['date'])
df_subway = df_subway.drop(['on_num', 'off_num'], axis=1)
df_subway


,date,total
0,2019-01-01,6813453
1,2019-01-02,15214002
2,2019-01-03,15716213
3,2019-01-04,16509281
4,2019-01-05,12092335
...,...,...
938,2021-07-27,11171030
939,2021-07-28,11122613
940,2021-07-29,11016238
941,2021-07-30,11247964


In [3]:
# df_subway, df_covid 합치기
df = pd.merge(df_subway, df_covid, how='outer', on='date')

# <Na>를 0으로 바꾸기
df = df.fillna(0)

# merge 할 때 지맘대로 data type이 float로 변경되서, 다시 int64로 변환
df['COUNT'] = df['COUNT'].astype('Int64')
df['total'] = df['total'].astype('Int64')

df.set_index('date', drop=False, inplace=True)

pd.set_option('display.max_row', 1500)
df

,date,total,COUNT
date,,,
2019-01-01,2019-01-01,6813453,0
2019-01-02,2019-01-02,15214002,0
2019-01-03,2019-01-03,15716213,0
2019-01-04,2019-01-04,16509281,0
2019-01-05,2019-01-05,12092335,0
2019-01-06,2019-01-06,8789266,0
2019-01-07,2019-01-07,15619172,0
2019-01-08,2019-01-08,15793382,0
2019-01-09,2019-01-09,15598474,0
